In [4]:
import pyaudio
import queue
import threading
import numpy as np
import math
import wave
import matplotlib.pyplot as plt
from scipy.io import wavfile


class MicArray:

    def __init__(self,device_index = None,rate = 16000,channels = 4,chunk_size = 1024):
        self.p = pyaudio.PyAudio()
        self.q = queue.Queue()
        self.thread_event = threading.Event()

        self.rate = rate
        self.channels = channels
        self.chunk_size = chunk_size
        
        if device_index =s= None:
            for i in range(self.p.get_device_count()):
                dev = self.p.get_device_info_by_index(i)
                name = dev['name'].encode('utf-8')
                print(i, name, dev['maxInputChannels'], dev['maxOutputChannels'])
                if dev['maxInputChannels'] == self.channels:
                    print('Use {}'.format(name))
                    device_index = i
                    break

                
        self.stream = self.p.open(
            input_device_index = device_index,
            start = False,
            format=pyaudio.paInt16,
            channels=self.channels,
            rate=self.rate,
            input=True,
            frames_per_buffer=self.chunk_size,
            stream_callback=self._callback
        )
               
    def _callback(self,input_data,frame_count,time_info,status_flag):
        self.q.put(input_data)
        return (None,pyaudio.paContinue)
    
    def start(self):
        self.q.queue.clear()
        self.stream.start_stream()
    
    def stop(self):
        self.thread_event.set()
        self.stream.stop_stream()
        self.q.put('')

    def read_mic_data(self):
        self.thread_event.clear()
        while not self.thread_event.is_set():
            frames = self.q.get()
            if not frames:
                break
            
            frames = np.frombuffer(frames,dtype = 'int16')
            yield frames

    def __enter__(self):
        self.start()
        return self

    def __exit__(self,exception_type,exception_value,traceback):
        if exception_value:
            return False
        self.stop()
        

def gccphat(sig,refsig,fs,max_tau,interp=1):
    
    n = sig.shape[0] + refsig.shape[0]

    SIG = np.fft.rfft(sig, n=n)
    REFSIG = np.fft.rfft(refsig, n=n)
    R = SIG * np.conj(REFSIG)

    cc = np.fft.irfft(R / np.abs(R), n=(interp * n))
    
    max_shift = np.minimum(int(interp * fs * max_tau), int(interp * n/2))
    
    cc = np.concatenate((cc[-max_shift:], cc[:max_shift+1]))
    
    shift = np.argmax(np.abs(cc)) - max_shift
    
    tau = shift / float(interp * fs)
    
    return tau

In [3]:
def find_direction_gccphat(data_frame,fs,tau_mic_dist):
    tau1 = gccphat(data_frame[0::4],data_frame[2::4],fs=fs,max_tau=tau_mic_dist)
    tau2 = gccphat(data_frame[1::4],data_frame[3::4],fs=fs,max_tau=tau_mic_dist)
    
    theta1 = math.asin(tau1/tau_mic_dist) * 180 / math.pi
    theta2 = math.asin(tau2/tau_mic_dist) * 180 / math.pi
    
    if np.abs(theta1) < np.abs(theta2):
        if theta2 > 0:
            final_theta = (theta1 + 360) % 360
        else:
            final_theta = (180 - theta1)
    else:
        if theta1 < 0:
            final_theta = (theta2 + 360) % 360
        else:
            final_theta = (180 - theta2)

        final_theta = (final_theta + 90 + 180) % 360


    final_theta = (-final_theta + 120) % 360
    
    return final_theta




def mic_array_test():
    import signal
    import time
    
    distance = 0.13
    max_time = distance/343.2
    
    is_quit = threading.Event()
    
    def signal_handler(sig, num):
        is_quit.set()
        print('Exited')
    
    signal.signal(signal.SIGINT, signal_handler)
    
    
    with MicArray(channels=4) as mic:
        for chunk in mic.read_mic_data():
            direction = find_direction_gccphat(chunk,fs=mic.rate,tau_mic_dist=max_time)
            print(int(direction))
            
            if is_quit.is_set():
                break

    
def main():
    mic_array_test()
    
    
if __name__ == "__main__":
    main()

0 b'Microsoft Sound Mapper - Input' 2 0
1 b'Microphone (USB Camera-B4.09.24' 4 0
Use b'Microphone (USB Camera-B4.09.24'
30
30
30
30
30
30
30
30
300
30
30
30
30
30
30
30
300
30
300
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
300
30
30
30
30
30
30
30
300
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
300
300
300
30
30
30
30
300
30
30
30
30
300
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
300
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
300
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
300
30
30
30
30
30
30
Exited
30
